In [58]:
import nltk
from nltk.corpus import stopwords
import numpy as np
import operator
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
#Retrieve the messages associated to each sender
training_info = pd.read_csv("/Users/Narjes/Documents/Paris-Saclay University/Learning for Text and Graph Data/Data competition 2017/training_info.csv")
training_set = pd.read_csv("/Users/Narjes/Documents/Paris-Saclay University/Learning for Text and Graph Data/Data competition 2017/training_set.csv")
test_set = pd.read_csv("/Users/Narjes/Documents/Paris-Saclay University/Learning for Text and Graph Data/Data competition 2017/test_set.csv")
test_info = pd.read_csv("/Users/Narjes/Documents/Paris-Saclay University/Learning for Text and Graph Data/Data competition 2017/test_info.csv")

In [ ]:
#Create a message file for each sender
for k in range(len(training_set)):
    sender_msgs = []
    mid_list = training_set['mids'][k].split(' ')
    for i in range(len(mid_list)):
        mid_list[i] = int(mid_list[i])

    for j in range(len(training_info)):
        if training_info['mid'][j] in mid_list:
            if len(sender_msgs) == 0:
                sender_msgs = pd.DataFrame(training_info.loc[[j]])
            else:
                sender_msgs = sender_msgs.append(pd.DataFrame(training_info.loc[[j]]))
    print(k)
    file_name =  (training_set['sender'][k].split('@'))[0] + ".csv"
    sender_msgs.to_csv(file_name, encoding='utf-8')

In [59]:
submission_data = pd.DataFrame(columns = ('mid', 'recipients'))
### Retrieve the data of all senders
for i in range(len(test_set)):
    file_name =  (training_set['sender'][i].split('@'))[0] + ".csv"
    print(file_name)
    df = pd.read_csv(file_name)
    # Transform the message to bag of words
    # We can more processing on bodies to increase the relevance of the choosen words
    # Eliminate stop words
    for j in range(len(df)):
        bag = ' '.join([word for word in df['body'][j].split('\t')])
        df['body'][j] = ' '.join([word for word in bag.split(' ') if not (len(word) < 4 or  word in stopwords.words("english"))])
    # Retrieve the test message of the sender
    msgs_id = [int(mid) for mid in test_set['mids'][i].split(' ')]
    ind = 0                           
    for mid in msgs_id:
    # Apply tfidf
        recipients_corpus = []
        submission_dic = dict()
        corpus = [email for email in df['body']]
        # Get the message and add it to the corpus
        msg_test = test_info[test_info.mid == mid]
        test = msg_test['body'][msg_test.index[0]]
        test = ' '.join([word for word in test.split('\t')])
        test = ' '.join([word for word in test.split(' ') if not (len(word) < 4 or word in stopwords.words("english"))])
        corpus.append(test)
        # TFIDF
        vectorizer = TfidfVectorizer(min_df=1)
        matrix = vectorizer.fit_transform(corpus)
        recipients_list = []
        add_rec = []
        matrix_test = matrix[len(corpus) - 1 : len(corpus)]       
        cosine_similarities = linear_kernel(matrix_test, matrix).flatten()
        related_docs_indices = cosine_similarities.argsort()[:-32:-1]
        for k in range(len(related_docs_indices)):
            if related_docs_indices[k] == len(corpus) - 1:
                index = k
        related_docs_indices = np.delete(related_docs_indices, index)
        for idx in related_docs_indices:
            recipients_list.append(df['recipients'][idx].split(' '))
            #For each email
            dic_recp = dict()
            for rep_list in recipients_list:
                for em_add in rep_list:
                    if em_add not in dic_recp.keys():
                        dic_recp[em_add] = cosine_similarities[related_docs_indices[recipients_list.index(rep_list)]]
                    else:
                        dic_recp[em_add] += cosine_similarities[related_docs_indices[recipients_list.index(rep_list)]]
        sorted_recep = sorted(dic_recp.items(), key=operator.itemgetter(1))
        for rep_sim in reversed(sorted_recep[-11:-1]):
            add_rec.append(rep_sim[0])
        submission_dic['mid'] = [mid]
        sub_add_list =' '.join([add for add in add_rec])
#        print(sub_add_list)
        submission_dic['recipients'] = [sub_add_list]
        if submission_data.empty:
            submission_data = pd.DataFrame(submission_dic)
        else:
            submission_data = submission_data.append(pd.DataFrame(submission_dic))
#        print(submission_data)
submission_data.to_csv('submission.csv', index=False, encoding='utf-8')

karen.buckley.csv


C:\Users\Narjes\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


amr.ibrahim.csv
andrea.ring.csv
sylvia.hu.csv
phillip.platter.csv
richard.shapiro.csv
megan.parker.csv
david.forster.csv
mike.maggi.csv
justin.rostant.csv
marcus.nettelton.csv
kathleen.carnahan.csv
grace.rodriguez.csv
hunter.s.shively.csv
sandra.f.brawner.csv
l..nicolay.csv
mark.whitt.csv
james.derrick.csv
darrell.schoolcraft.csv
l..denton.csv
cheryl.johnson.csv
scott.neal.csv
chris.germany.csv
eric.bass.csv
larry.f.campbell.csv
lynn.blair.csv
nancy.sellers.csv
harry.kingerski.csv
m..forney.csv
stacey.w.white.csv
rick.buy.csv
matt.smith.csv
dutch.quigley.csv
greg.piper.csv
stanley.horton.csv
liz.taylor.csv
jason.williams.csv
taylor.csv
alex.csv
mary.cook.csv
m..schmidt.csv
joe.stepenovitch.csv
mike.carson.csv
paul.kaufman.csv
sheila.glover.csv
jane.tholt.csv
monika.causholli.csv
jim.schwieger.csv
andy.zipper.csv
janel.guerrero.csv
kim.ward.csv
lisa.mellencamp.csv
beth.cherry.csv
david.port.csv
kevin.m.presto.csv
mike.grigsby.csv
julie.armstrong.csv
wsmith.csv
martin.cuilla.csv
phillip.